In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [3]:
X, y

(array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
        [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
        [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
        [1.00e+00, 5.00e+00, 6.00e+02, 1.00e+00],
        [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
        [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
        [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
        [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
        [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
        [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

In [4]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [5]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [6]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

In [7]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [8]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [9]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 100) == 0:
            print(i, W, err)
    return W

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

Модифицируем функцию так, чтобы в неё попадали не 0 или 1, а близкие к ним значения.

In [10]:
def calc_logloss_mod(y, y_pred):
    y_pred_res = np.where(y_pred == 1, y_pred - 1e-07, np.where(y_pred == 0, y_pred + 1e-07, y_pred))
    err = - np.mean(y * np.log(y_pred_res) + (1.0 - y) * np.log(1.0 - y_pred_res))
    return err

Для проверки изменим вектор предсказанных значений, добавив в него нули и единицы

In [11]:
y_pred_2 = np.array([0., 0., 0.73370977, 0.45678552, 1.,
       0.57185484, 0.76730523, 1. , 0., 0.690758 ])
calc_logloss_mod(y, y_pred_2)

3.4639213705337264

Исходная функция возвращает nan

In [12]:
calc_logloss(y, y_pred_2)

C:\Users\User\AppData\Local\Temp\ipykernel_2068\851670199.py:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
C:\Users\User\AppData\Local\Temp\ipykernel_2068\851670199.py:2: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


nan

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [13]:
def eval_model(X, y, verbose=False, alpha=1e-4, tol=0.00001): 
    view_ind = 10**(-np.log10(tol)-2) if -np.log10(tol)-2 >= 1 else 1 # задаём параметр кратности вывода промежуточных результатов
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    min_err = np.inf 
    n_iter = 0
    stop_check = True
    errors = []
    while stop_check:
        n_iter += 1
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss_mod(y, y_pred)
        errors.append(err)
        if min_err - err > tol:  # контролируем текущее значение ошибки
            min_err = err
        else:  # если снижение прекратилось, останавливаемся.
            print(
                f'Stop descent. Iteration: {n_iter}, weights: {W}, logloss: {min_err}')
            stop_check = False
        dQ = 1/n * X.T @ (y_pred - y)
        W -= alpha * dQ
        if verbose:
            if n_iter % view_ind == 0:
                print(n_iter, W, err)
    return W, min_err, n_iter

In [14]:
W = eval_model(X_st, y, verbose=True, alpha=0.6, tol=0.00001)

1000 [-8.22705654 -1.19179224 -1.63103389  7.14617725] 0.2597559875895107
2000 [-11.81693228  -1.47202752  -2.74842535   9.99506855] 0.2221605978051023
3000 [-14.60325068  -1.70958893  -3.58457685  12.24861365] 0.19942508537234382
4000 [-16.96660182  -1.91673038  -4.28184084  14.16798183] 0.18306205104868484
5000 [-19.03415014  -2.10036075  -4.88520289  15.84885732] 0.1705473305742659
Stop descent. Iteration: 5449, weights: [-19.88389849  -2.17629563  -5.13166996  16.53977688], logloss: 0.16583588446774902


Подберём лучший параметр $\alpha$ для модели.

In [15]:
def get_best_params(X, y, args):
    best_params = []
    for arg in args:
        W,err,n_iter = eval_model(X, y, alpha=arg)
        best_params.append((arg,err,n_iter))
    best_params.sort(key = lambda x:x[1])
    print(f'best alpha: {best_params[0][0]}, \nresults:\nerr: {best_params[0][1]}, \nn_iter: {best_params[0][2]}')
    return best_params[0]

In [42]:
alphas = [1, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001, 0.000005, 0.000001, 0.0000005, 0.0000001]
best_param = get_best_params(X_st, y, alphas)
best_param

Stop descent. Iteration: 2, weights: [ 0.11732727 -1.58914029  0.62030812  0.95688359], logloss: 1.1785958344356262
Stop descent. Iteration: 5588, weights: [-18.35029116  -2.03943006  -4.68623881  15.29282887], logloss: 0.17451429964096338
Stop descent. Iteration: 5507, weights: [-7.85409116 -1.16596006 -1.51053115  6.85817166], logloss: 0.2644646885841898
Stop descent. Iteration: 6217, weights: [-5.68112112 -1.03751043 -0.78050594  5.23775616], logloss: 0.29772674984784453
Stop descent. Iteration: 5665, weights: [-1.51077461 -0.92062082  0.7677089   2.49733437], logloss: 0.4113212951463094
Stop descent. Iteration: 2705, weights: [-0.28749979 -0.77956547  0.97673079  1.56687312], logloss: 0.46890406876264856
Stop descent. Iteration: 1674, weights: [ 0.24314054 -0.71650811  0.70726759  1.27956464], logloss: 0.5115007726748093
Stop descent. Iteration: 1980, weights: [ 0.30188575 -0.69945257  0.66235975  1.28918087], logloss: 0.5203618833145909
Stop descent. Iteration: 6102, weights: [ 0.

(0.5, 0.17451429964096338, 5588)

Лучший параметр скорости обучения в данном примере $\alpha = 0.5$.

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [16]:
def calc_pred_proba(W, X): 
    pred_proba = sigmoid(np.dot(X, W))
    return pred_proba

In [17]:
W, err, iteration = eval_model(X_st, y, alpha=0.5, verbose=True)

1000 [-7.45770662 -1.13952545 -1.38116181  6.55470777] 0.2698163959605315
2000 [-10.74690483  -1.38416322  -2.42154793   9.13598506] 0.2321438079024516
3000 [-13.27474073  -1.59511144  -3.18818218  11.17211031] 0.20973740486390463
4000 [-15.42904321  -1.78154353  -3.82925471  12.91883838] 0.19343288558614483
5000 [-17.32873106  -1.94875871  -4.38792531  14.46232766] 0.1807541199248279
Stop descent. Iteration: 5588, weights: [-18.35029116  -2.03943006  -4.68623881  15.29282887], logloss: 0.17451429964096338


In [18]:
y_pred_proba = calc_pred_proba(W, X_st)
y_pred_proba

array([3.76064786e-01, 8.02194945e-02, 9.99967427e-01, 6.56473375e-05,
       8.21509735e-01, 2.03213448e-01, 9.99995255e-01, 5.88433938e-02,
       4.94246689e-01, 9.99635138e-01])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [19]:
def calc_pred(W, X, prob_lim=0.5):
    pred_proba = sigmoid(np.dot(X, W))
    pred = np.zeros_like(pred_proba)
    for idx, prob in enumerate(pred_proba):
        if prob > prob_lim:
            pred[idx] = 1
    return pred

In [20]:
y_pred = calc_pred(W, X_st)
y_pred

array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.])

5. Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

**Accuracy**

In [24]:
def my_accuracy(real, pred):
    all_results = len(real)
    trues = 0
    for i in range(all_results):
        trues += int(real[i] == pred[i])
    return trues / all_results

In [25]:
my_accuracy(y, y_pred)

0.9

Проверка:

In [26]:
accuracy_score(y, y_pred)

0.9

**Матрица ошибок**

In [27]:
def my_confusion_matrix(real, pred):
    n_classes = len(np.unique(real))
    all_results = len(real)
    confusion_matrix = np.zeros((n_classes, n_classes), dtype='int')
    for i in range(all_results):
        confusion_matrix[int(real[i])][int(pred[i])] += 1
    return confusion_matrix

In [28]:
my_confusion_matrix(y, y_pred)

array([[5, 0],
       [1, 4]])

Проверка:

In [29]:
confusion_matrix(y, y_pred)

array([[5, 0],
       [1, 4]], dtype=int64)

**Точность**

In [30]:
def my_precision(real, pred):
    tn, fp, fn, tp = my_confusion_matrix(real, pred).ravel()
    return tp / (tp+fp)

In [31]:
my_precision(y, y_pred)

1.0

In [32]:
precision_score(y, y_pred)

1.0

**Полнота**

In [33]:
def my_recall(real, pred):
    tn, fp, fn, tp = my_confusion_matrix(real, pred).ravel()
    return tp / (tp+fn)

In [34]:
my_recall(y, y_pred)

0.8

In [35]:
recall_score(y, y_pred)

0.8

**F1-score**

In [36]:
def my_f1(real, pred, beta=1):
    f_score = (1+beta**2) * my_precision(real, pred) * my_recall(y,y_pred) / (beta**2*my_precision(real, pred) + my_recall(y,y_pred))
    return f_score

In [37]:
my_f1(y, y_pred)

0.888888888888889

In [38]:
f1_score(y, y_pred)

0.888888888888889

6. Могла ли модель переобучиться? Почему?

Модель логистической регрессии склонна к переобучению в следствие того, что мы используем сигмоиду, чтобы преобразовать в вероятность предсказания линейной модели. А т.к. сигмоида не имеет максимума и минимума, а только асимптоты в 0 и 1, градиентный спуск не может достичь оптимального решения с помощью градиентных шагов, доводя веса до все более экстремальных значений, пытаясь достичь нулевых потерь. При большой размерности данных вероятность этого еще больше увеличивается.